In [101]:
import extract
import pandas as pd
import os
import geopandas as gpd

In [102]:
import imp
imp.reload(extract)

<module 'extract' from 'd:\\Users\\User\\Downloads\\RA Work\\acs\\extract.py'>

In [103]:
# open state_fips.csv
state_fips = pd.read_csv('state_fips.csv')
state_fips.head()

,state,code,abbreviation
0,Alabama,1,AL
1,Alaska,2,AK
2,Arizona,4,AZ
3,Arkansas,5,AR
4,California,6,CA


In [117]:
# for every folder in raw_tract_data
for folder in os.listdir('raw_tract_data'):
    # split folder into list by underscore
    folder_tokens = folder.split('_')
    
    # get state column from state_fips where code is third item in list as int
    state = state_fips[state_fips['code'] == int(folder_tokens[2])]['state'].values[0].lower()

    # if last item in folder is 'tract' year is 2000, otherwise it is 20 + last two digits
    year = '2020' if folder_tokens[-1] == 'tract' else '20' + folder_tokens[-1][-2:]

    # if tract file already exists, skip
    if os.path.exists(f'tract_polygons/{state}_tracts_{year}.csv'):
        continue

    # get the filepath of the shp file in the folder with the same name
    shp_file = os.path.join('raw_tract_data', folder, folder + '.shp')

    tract_geos = extract.read_tracts(shp_file)
    
    # save as acs\tract_polygons\berkeley_tracts_2010.csv
    tract_geos.to_csv(f'tract_polygons/{state}_tracts_{year}.csv', index=False)

In [46]:
# for alaska upper (weird KML)
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
districts = gpd.read_file('raw_geo_data\\alaska_upper_2022.kml', driver='KML')
# rename Name to district
districts = districts.rename(columns={'Name': 'district'})
# only keep district and geometry
districts = districts[['district', 'geometry']]
# only keep the part of the district field in the form 0200A
districts['district'] = districts['district'].str.extract(r'(\d{4}[A-Z])')
# only keep the last letter
districts['district'] = districts['district'].str[-1]
# save to geo_polygons folder as alaska_upper_districts_2022.csv
# districts.to_csv('geo_polygons/alaska_upper_districts_2022.csv', index=False)

In [125]:
# for SHP
# read in shp
districts = gpd.read_file(r'raw_geo_data\Minneapolis_Park_Commissioner_Districts_2022\Minneapolis_Park_Commissioner_Districts_2022.shp')
districts

,OBJECTID,MPLSPARK,Shape__Are,Shape__Len,geometry
0,1,1,3.284170e+08,98390.029893,"POLYGON ((-93.22693 45.03567, -93.22693 45.033..."
1,2,2,2.836905e+08,81825.208696,"POLYGON ((-93.31950 45.05125, -93.31823 45.051..."
2,3,3,1.594340e+08,69333.502519,"POLYGON ((-93.22573 44.96616, -93.22286 44.963..."
3,4,4,1.928964e+08,97013.396958,"POLYGON ((-93.26800 44.98965, -93.26765 44.989..."
4,5,5,3.401534e+08,104630.772827,"POLYGON ((-93.21003 44.95469, -93.20776 44.953..."
5,6,6,2.987371e+08,88954.577536,"POLYGON ((-93.32251 44.96048, -93.32252 44.960..."


In [126]:
# only keep district and geometry
district_name = 'MPLSPARK'
year=2022
desc='minneapolis_park_commissioner'
districts = districts[[district_name, 'geometry']]
# rename to district and convert to int
districts = districts.rename(columns={district_name: 'district'})
districts['district'] = districts['district'].astype(int)

In [124]:
# save to geo_polygons folder
districts.to_csv(f'geo_polygons/{desc}_{year}.csv', index=False)

In [131]:
# open csv
districts = pd.read_csv(f'raw_geo_data\Oakland_CouncilDistrict_20240609.csv')
districts

,the_geom,FULLNAME,NAME,ID,ACTIONID,AGENCY,RESPONSE,MESSAGE,COUNCILMEM,EMAIL,LINK,SHAPE_AREA,SHAPE_LEN
0,MULTIPOLYGON (((-122.27879380063283 37.8272020...,COUNCIL DISTRICT 3,CCD3,9,NaN,NaN,NaN,0,Lynette Gibson McElhaney,lmcelhaney@oaklandnet.com,http://www2.oaklandnet.com/Government/o/CityCo...,3.673364e+08,95531.542967
1,MULTIPOLYGON (((-122.24485881083257 37.7791330...,COUNCIL DISTRICT 2,CCD2,22,NaN,NaN,NaN,0,Abel Guillen,aguillen@oaklandnet.com,NaN,1.103135e+08,71339.408608
2,MULTIPOLYGON (((-122.17859682315829 37.7552280...,COUNCIL DISTRICT 7,CCD7,13,NaN,NaN,NaN,0,Larry Reid,lreid@oaklandnet.com,http://www2.oaklandnet.com/Government/o/CityCo...,8.823895e+08,214965.999251
3,MULTIPOLYGON (((-122.21483881680408 37.7614630...,COUNCIL DISTRICT 5,CCD5,11,NaN,NaN,NaN,0,Noel Gallo,ngallo@oaklandnet.com,http://www2.oaklandnet.com/Government/o/CityCo...,1.168510e+08,77753.865010
4,MULTIPOLYGON (((-122.17840382327432 37.7551160...,COUNCIL DISTRICT 6,CCD6,31,NaN,NaN,NaN,0,Desley Brooks,dbrooks@oaklandnet.com,http://www2.oaklandnet.com/Government/o/CityCo...,2.147994e+08,103471.879993
5,MULTIPOLYGON (((-122.27879380063283 37.8272020...,COUNCIL DISTRICT 1,CCD1,15,NaN,NaN,NaN,0,Dan Kalb,dkalb@oaklandnet.com,http://www2.oaklandnet.com/Government/o/CityCo...,2.448953e+08,119162.886117
6,MULTIPOLYGON (((-122.16869181865106 37.7992670...,COUNCIL DISTRICT 4,CCD4,3,NaN,NaN,NaN,0,Anne Campbell Washington,ACampbell-Washington@oaklandnet.com,NaN,2.374677e+08,109742.881135


In [132]:
# only keep the_geom and NAME
districts = districts[['the_geom', 'NAME']]

# rename the_geom to geometry and NAME to district
districts = districts.rename(columns={'the_geom': 'geometry', 'NAME': 'district'})

# only keep the last char of NAME and convert to int
districts['district'] = districts['district'].str[-1].astype(int)

In [133]:
# save to geo_polygons folder
districts.to_csv(f'geo_polygons/oakland_city_council_2022.csv', index=False)

,geometry,district
0,MULTIPOLYGON (((-122.27879380063283 37.8272020...,3
1,MULTIPOLYGON (((-122.24485881083257 37.7791330...,2
2,MULTIPOLYGON (((-122.17859682315829 37.7552280...,7
3,MULTIPOLYGON (((-122.21483881680408 37.7614630...,5
4,MULTIPOLYGON (((-122.17840382327432 37.7551160...,6
5,MULTIPOLYGON (((-122.27879380063283 37.8272020...,1
6,MULTIPOLYGON (((-122.16869181865106 37.7992670...,4
